In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS

from sklearn.tree import (DecisionTreeClassifier,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             log_loss)
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
from pathlib import Path
from tqdm.notebook import tqdm

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.utils import resample

In [2]:
# NUM_POSTE   : numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   : nom usuel du poste
# LAT         : latitude, négative au sud (en degrés et millionièmes de degré)
# LON         : longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        : altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJ    : date de la mesure (année mois jour)
# RR          : quantité de précipitation tombée en 24 heures (de 06h FU le jour J à 06h FU le jour J+1). La valeur relevée à J+1 est affectée au jour J (en mm et 1/10)
# TN          : température minimale sous abri (en °C et 1/10)
# HTN         : heure de TN (hhmm)
# TX          : température maximale sous abri (en °C et 1/10)
# HTX         : heure de TX (hhmm)
# TM          : moyenne quotidienne des températures horaires sous abri (en °C et 1/10)
# TNTXM       : moyenne quotidienne (TN+TX)/2 (en °C et 1/10)
# TAMPLI      : amplitude thermique quotidienne : écart entre TX et TN quotidiens (TX-TN) (en °C et 1/10)
# TNSOL       : température quotidienne minimale à 10 cm au-dessus du sol (en °C et 1/10)
# TN50        : température quotidienne minimale à 50 cm au-dessus du sol (en °C et 1/10)
# DG          : durée de gel sous abri (T ≤ 0°C) (en mn)
# FFM         : moyenne quotidienne de la force du vent moyenné sur 10 mn, à 10 m (en m/s et 1/10)
# FF2M        : moyenne quotidienne de la force du vent moyenné sur 10 mn, à 2 m (en m/s et 1/10)
# FXY         : maximum quotidien de la force maximale horaire du vent moyenné sur 10 mn, à 10 m (en m/s et 1/10)
# DXY         : direction de FXY (en rose de 360)
# HXY         : heure de FXY (hhmm)
# FXI         : maximum quotidien de la force maximale horaire du vent instantané, à 10 m (en m/s et 1/10)
# DXI         : direction de FXI (en rose de 360)
# HXI         : heure de FXI (hhmm)
# FXI2        : maximum quotidien de la force maximale horaire du vent instantané, à 2 m (en m/s et 1/10)
# DXI2        : direction de FXI2 (en rose de 360)
# HXI2        : heure de FXI2 (hhmm)
# FXI3S       : maximum quotidien de la force maximale horaire du vent moyenné sur 3 s, à 10 m (en m/s et 1/10)
# DXI3S       : direction de FXI3S (en rose de 360)
# HXI3S       : heure de FXI3S (hhmm)
# DRR         : durée des précipitations (en mn)

# A chaque donnée est associé un code qualité (ex: T;QT) :
#  9 : donnée filtrée (la donnée a passé les filtres/contrôles de premiers niveaux)
#  0 : donnée protégée (la donnée a été validée définitivement par le climatologue)
#  1 : donnée validée (la donnée a été validée par contrôle automatique ou par le climatologue)
#  2 : donnée douteuse en cours de vérification (la donnée a été mise en doute par contrôle automatique)
 
# D'une façon générale, les valeurs fournies sont données avec une précision qui correspond globalement à la résolution de l'appareil de mesure de la valeur.
# Toutefois, il peut arriver, pour des raisons techniques de stokage ou d'extraction des valeurs, que cette règle ne soit pas respectée.
# Du fait d'arrondis, il peut ponctuellement arriver que des valeurs de base à un pas de temps inférieur (par exemple données minutes) ne soient pas exactement cohérentes avec leurs correspondants sur un pas de temps supérieur (par exemple données horaires).

In [65]:
df = pd.read_feather('../data/merged_meteo_red_days_from_20170101.feather')
print(df.columns)
df.head()

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_MONTPELLIER', 'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG',
       'TN_TOULOUSE', 'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE',
       'TX_MONTPELLIER', 'TX_NANTES', 'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG',
       'TX_TOULOUSE', 'TM_BORDEAUX', 'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE',
       'TM_MONTPELLIER', 'TM_NANTES', 'TM_NICE', 'TM_PARIS', 'TM_STRASBOURG',
       'TM_TOULOUSE', 'TNTXM_BORDEAUX', 'TNTXM_LILLE', 'TNTXM_LYON',
       'TNTXM_MARSEILLE', 'TNTXM_MONTPELLIER', 'TNTXM_NANTES', 'TNTXM_NICE',
       'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE', 'TAMPLI_BORDEAUX',
       'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE', 'TAMPLI_MONTPELLIER',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_MONTPELLIER', 'FFM_NANTES', 'FFM_NICE',
       'FFM_PARIS', 'FFM

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-2.9,...,1.4,0.0,0.2,0.0,1.6,0.0,0.2,0.0,0.0,False
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-1.9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.0,0.0,False
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-3.7,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,-1.9,...,2.4,1.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,True
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [66]:
# Drop non TN/TX/TM cols
# cols_to_keep = [col for col in df.columns if  'TN_' in col or  'TX_' in col in col or col == 'AAAAMMJJ' or col == 'is_red_day']
# df = df[cols_to_keep]

In [67]:
# Remove days after 20240311
df = df[df['AAAAMMJJ'] < 20240311]
df.tail()

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
2621,20240306,6.2,NaN,3.9,3.3,4.9,1.8,6.6,5.4,3.1,...,NaN,0.0,0.0,0.0,0.2,0.0,0.0,2.0,0.0,True
2622,20240307,5.7,NaN,0.3,0.8,0.5,4.0,7.4,3.8,3.6,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2623,20240308,8.9,NaN,4.8,7.0,8.7,6.9,6.9,3.6,-0.8,...,NaN,0.4,8.0,13.1,1.4,8.1,0.0,0.0,12.9,False
2624,20240309,7.5,NaN,5.9,7.7,9.9,6.2,8.2,6.4,2.3,...,NaN,15.1,69.4,21.8,8.5,40.3,6.5,0.0,0.0,False
2625,20240310,3.3,NaN,5.7,9.2,6.3,0.9,8.4,7.1,6.4,...,NaN,4.4,3.6,0.0,7.5,23.8,0.0,7.3,0.2,False


In [68]:
# Count nan values and show nan cols:

print(df.isna().sum())
pd.set_option('display.max_rows', None) 
df.columns[df.isna().sum() > 0]

AAAAMMJJ                0
TN_BORDEAUX             0
TN_LILLE               70
TN_LYON                 0
TN_MARSEILLE            0
TN_MONTPELLIER        365
TN_NANTES               0
TN_NICE                 0
TN_PARIS                0
TN_STRASBOURG           0
TN_TOULOUSE             0
TX_BORDEAUX             0
TX_LILLE               70
TX_LYON                 0
TX_MARSEILLE            0
TX_MONTPELLIER        365
TX_NANTES               0
TX_NICE                 0
TX_PARIS                0
TX_STRASBOURG           0
TX_TOULOUSE             0
TM_BORDEAUX             0
TM_LILLE               70
TM_LYON                 0
TM_MARSEILLE            5
TM_MONTPELLIER        365
TM_NANTES               0
TM_NICE                 1
TM_PARIS                0
TM_STRASBOURG           0
TM_TOULOUSE             0
TNTXM_BORDEAUX          0
TNTXM_LILLE            70
TNTXM_LYON              0
TNTXM_MARSEILLE         0
TNTXM_MONTPELLIER     365
TNTXM_NANTES            0
TNTXM_NICE              0
TNTXM_PARIS 

Index(['TN_LILLE', 'TN_MONTPELLIER', 'TX_LILLE', 'TX_MONTPELLIER', 'TM_LILLE',
       'TM_MARSEILLE', 'TM_MONTPELLIER', 'TM_NICE', 'TNTXM_LILLE',
       'TNTXM_MONTPELLIER', 'TAMPLI_LILLE', 'TAMPLI_MONTPELLIER',
       'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_MARSEILLE', 'FFM_MONTPELLIER',
       'FFM_PARIS', 'FFM_STRASBOURG', 'FFM_TOULOUSE', 'RR_LILLE',
       'RR_MONTPELLIER'],
      dtype='object')

In [69]:
df.columns

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_MONTPELLIER', 'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG',
       'TN_TOULOUSE', 'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE',
       'TX_MONTPELLIER', 'TX_NANTES', 'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG',
       'TX_TOULOUSE', 'TM_BORDEAUX', 'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE',
       'TM_MONTPELLIER', 'TM_NANTES', 'TM_NICE', 'TM_PARIS', 'TM_STRASBOURG',
       'TM_TOULOUSE', 'TNTXM_BORDEAUX', 'TNTXM_LILLE', 'TNTXM_LYON',
       'TNTXM_MARSEILLE', 'TNTXM_MONTPELLIER', 'TNTXM_NANTES', 'TNTXM_NICE',
       'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE', 'TAMPLI_BORDEAUX',
       'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE', 'TAMPLI_MONTPELLIER',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_MONTPELLIER', 'FFM_NANTES', 'FFM_NICE',
       'FFM_PARIS', 'FFM

In [70]:
# drop montepellier
df.drop(columns=[c for c in df.columns if 'MONTPELLIER' in c], axis=1, inplace=True)

In [71]:
print(df.isna().sum())


AAAAMMJJ              0
TN_BORDEAUX           0
TN_LILLE             70
TN_LYON               0
TN_MARSEILLE          0
TN_NANTES             0
TN_NICE               0
TN_PARIS              0
TN_STRASBOURG         0
TN_TOULOUSE           0
TX_BORDEAUX           0
TX_LILLE             70
TX_LYON               0
TX_MARSEILLE          0
TX_NANTES             0
TX_NICE               0
TX_PARIS              0
TX_STRASBOURG         0
TX_TOULOUSE           0
TM_BORDEAUX           0
TM_LILLE             70
TM_LYON               0
TM_MARSEILLE          5
TM_NANTES             0
TM_NICE               1
TM_PARIS              0
TM_STRASBOURG         0
TM_TOULOUSE           0
TNTXM_BORDEAUX        0
TNTXM_LILLE          70
TNTXM_LYON            0
TNTXM_MARSEILLE       0
TNTXM_NANTES          0
TNTXM_NICE            0
TNTXM_PARIS           0
TNTXM_STRASBOURG      0
TNTXM_TOULOUSE        0
TAMPLI_BORDEAUX       0
TAMPLI_LILLE         70
TAMPLI_LYON           0
TAMPLI_MARSEILLE      0
TAMPLI_NANTES   

In [72]:
df.dropna(inplace=True)

Feature engineering

In [73]:
# Add a is week day feature:
daydt = pd.to_datetime(df['AAAAMMJJ'], format='%Y%m%d').dt
df['is_week_day'] = daydt.dayofweek < 5

In [82]:
# Add the number of elapsed red days for the current year
from datetime import datetime

df['is_red_day'] = df['is_red_day'].astype(int)
df['elapsed_red_days'] = df.groupby(daydt.year)['is_red_day'].cumsum()
df['last_day_was_red'] = df['is_red_day'].shift(1).fillna(0).astype(int)

# end_of_march = datetime(daydt.year if daydt.month < 3 else daydt.year + 1, 3, 31)
# fix the code below
# end_of_march = daydt.apply(lambda x: x.replace(year=x.year + 1, month=3, day=31) if x.month >= 3 else x.replace(month=3, day=31))
# df['days_until_end_of_march'] = (end_of_march - daydt).dt.days
# df['days_until_end_of_march']

In [83]:
# df['is_thursday_or_friday'] = daydt.dayofweek.isin([3, 4])

In [84]:
# Remove data between  01/04 and 01/11
df = df[~((daydt.month >= 4) & (daydt.month <= 10))]

In [85]:
df.columns

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG', 'TN_TOULOUSE',
       'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE', 'TX_NANTES',
       'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG', 'TX_TOULOUSE', 'TM_BORDEAUX',
       'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE', 'TM_NANTES', 'TM_NICE',
       'TM_PARIS', 'TM_STRASBOURG', 'TM_TOULOUSE', 'TNTXM_BORDEAUX',
       'TNTXM_LILLE', 'TNTXM_LYON', 'TNTXM_MARSEILLE', 'TNTXM_NANTES',
       'TNTXM_NICE', 'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE',
       'TAMPLI_BORDEAUX', 'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_NANTES', 'FFM_NICE', 'FFM_PARIS',
       'FFM_STRASBOURG', 'FFM_TOULOUSE', 'RR_BORDEAUX', 'RR_LILLE', 'RR_LYON',
       'RR_MARSEILLE', 'RR_NANTES', 'RR_NICE', 'RR_PARI

Learning

In [86]:
#  show values count in y
df['is_red_day'].value_counts()

is_red_day
0    895
1    149
Name: count, dtype: int64

In [87]:
def evaluate(model, X, y, plot_roc_rurve=False):
    y_pred = model.predict(X)
    print(f'Accuracy: {accuracy_score(y, y_pred)}')
    print('Precision', precision_score(y, y_pred))
    print('Recall', recall_score(y, y_pred))
    print('F1 score', f1_score(y, y_pred))
    display(confusion_table(y, y_pred))

    if plot_roc_rurve:
        y_pred_proba = model.predict_proba(X)[:,1]
        fpr, tpr, _ = roc_curve(y, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        fig = px.area(x=fpr, y=tpr, title=f'ROC Curve (AUC={roc_auc:.4f})',
                    labels=dict(x='False Positive Rate', y='True Positive Rate'),
                    width=700, height=500
                    )
        fig.show()

In [88]:
X = df.drop(['is_red_day', 'AAAAMMJJ'], axis=1)
todrop = []
for c in X.columns:
    if 'TAMPLI' in c or 'RR' in c or 'FFM' in c or 'TM' in c or 'TNTXM' in c:
        todrop.append(c)
X = X.drop(todrop, axis=1)
y = df['is_red_day']


X_train_val, X_test, y_train_val, y_test = skm.train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
# Over/undersamping
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN


X_train, X_val, y_train, y_val = skm.train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Not working better
# rus = RandomUnderSampler(random_state=42)
# X_train_resampl, y_train_resampl = rus.fit_resample(X_train, y_train)


# smote = SMOTE(random_state=42)
# X_train_resampl, y_train_resampl = smote.fit_resample(X_train, y_train)

# smote_enn = SMOTEENN(random_state=42)
# X_train_resampl, y_train_resampl = smote_enn.fit_resample(X_train, y_train)


In [90]:
# Train with xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=20, max_depth=8, random_state=0, max_delta_step=1,scale_pos_weight=20)
xgb.fit(X_train, y_train)

print('Train')
evaluate(xgb, X_train, y_train)

print('Validation')
evaluate(xgb, X_val, y_val, plot_roc_rurve=True)

Train
Accuracy: 0.9985029940119761
Precision 0.9894736842105263
Recall 1.0
F1 score 0.9947089947089947


Truth,0,1
Predicted,,
0,573,1
1,0,94


Validation
Accuracy: 0.8802395209580839
Precision 0.5952380952380952
Recall 0.8928571428571429
F1 score 0.7142857142857143


Truth,0,1
Predicted,,
0,122,17
1,3,25


In [47]:
# Cross val for hp tuning
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from joblib import parallel_backend

xgb = XGBClassifier(random_state=0)

# Define the parameter grid
param_grid = {
    'n_estimators': [20, 100, 200],
    'learning_rate': [0.1, 0.5, 1],
    'max_depth': [4, 8, 16],
    'min_child_weight': [1, 5, 10],
    'max_delta_step': [1, None],
    'gamma': [0, 0.1, 0.5],
    # 'subsample': [0.8, 1.0],
    # 'colsample_bytree': [0.8, 1.0],
    'scale_pos_weight': [1,20],  # Based on class imbalance ratio
    
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='f1', cv=3, verbose=10, n_jobs=-1)
with parallel_backend('multiprocessing'):
    grid_search.fit(X_train, y_train)

# Print the best parameters and evaluate
print("Best parameters found: ", grid_search.best_params_)

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
y_pred_val = best_model.predict(X_val)
print(classification_report(y_val, y_pred_val))
evaluate(best_model, X_val, y_val, plot_roc_rurve=True)


Fitting 3 folds for each of 972 candidates, totalling 2916 fits
Best parameters found:  {'gamma': 0.5, 'learning_rate': 0.1, 'max_delta_step': None, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 100, 'scale_pos_weight': 20}
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       139
           1       0.67      0.64      0.65        28

    accuracy                           0.89       167
   macro avg       0.80      0.79      0.79       167
weighted avg       0.88      0.89      0.89       167

Accuracy: 0.8862275449101796
Precision 0.6666666666666666
Recall 0.6428571428571429
F1 score 0.6545454545454545


Truth,0,1
Predicted,,
0,130,9
1,10,18


In [21]:
# # Explore incorrect predictions

# y_pred_val = xgb.predict(X_val)
# incorrect_rows_indices = X_val[y_pred_val != y_val].index

# incorrect_y_pred = xgb.predict(X)
# incorrect_preds = df.iloc[incorrect_rows_indices].copy()
# incorrect_preds['y_pred'] = incorrect_y_pred[incorrect_rows_indices]

# assert all(incorrect_preds['y_pred'] != incorrect_preds['is_red_day']), 'y_pred should be different from is_red_day'
# incorrect_preds

In [38]:
# Explore incorrect predictions

y_pred_val = best_model.predict(X_val)
incorrect_rows_indices = X_val[y_pred_val != y_val].index

y_pred = best_model.predict(X)
df_copy = df.copy()
df_copy['y_pred'] = y_pred

df_copy[(df_copy['is_red_day'] == 1) & (0 == df_copy['y_pred'])] 

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,TN_TOULOUSE,...,RR_MARSEILLE,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day,is_week_day,elapsed_red_days,y_pred
82,20170324,3.0,6.4,6.0,10.8,2.4,11.3,7.8,6.9,3.0,...,0.2,0.2,4.6,3.6,0.0,12.7,1,True,19,0
332,20171129,1.9,0.5,2.1,4.0,-1.7,6.6,1.9,1.0,3.9,...,0.0,0.2,0.0,1.0,0.4,0.0,1,True,22,0
345,20171212,0.4,0.2,2.2,3.6,-2.0,6.1,1.8,1.5,4.4,...,0.0,3.0,0.0,0.0,0.0,0.0,1,True,25,0
407,20180212,-0.5,0.1,0.5,3.7,-2.4,4.2,0.7,-0.7,4.0,...,0.0,0.6,6.1,1.0,0.0,0.0,1,True,6,0
690,20181122,5.9,0.1,1.4,8.2,1.3,9.9,1.3,-0.6,5.0,...,2.0,2.2,5.0,0.4,0.0,0.0,1,True,16,0
786,20190226,4.2,2.6,1.9,6.8,2.8,9.9,6.8,-0.2,2.6,...,0.0,0.0,0.0,0.0,0.0,0.0,1,True,9,0
810,20190322,2.9,5.7,1.9,4.9,4.7,10.0,6.7,0.0,1.6,...,0.0,0.0,0.0,0.0,0.0,0.0,1,True,15,0
815,20190327,3.6,6.8,1.5,2.8,1.2,7.6,5.6,1.5,1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,1,True,17,0
816,20190328,4.4,8.1,2.7,2.8,3.2,8.4,7.1,1.1,2.1,...,0.0,0.0,0.0,0.0,0.0,0.0,1,True,18,0
1114,20200120,2.3,-1.7,-1.7,4.3,2.6,7.8,2.3,1.6,-2.7,...,0.4,0.0,0.0,0.0,0.0,0.0,1,True,1,0


Notes:
- vacances scolaires
- Vitesse et direction du vent : Le vent peut influencer le ressenti de froid, augmentant ainsi les besoins énergétiques.
- Historique de consommation : Intégrer un décalage temporel pour inclure la consommation des jours précédents peut être utile.
- Prévisions de consommation : Des estimations faites par EDF ou des modèles tiers peuvent être une variable clé.

In [23]:
Notes

NameError: name 'Notes' is not defined